# 1. Webscraping e criação do DataFrame

## Importando bibliotecas e arquivos

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
lista_empresa = pd.read_csv('lista_empresa.csv', sep=';')

In [3]:
ticks = list(lista_empresa['codigo'])
ticks = [tick.lower() for tick in ticks]
ticks

['aalr3',
 'abcb4',
 'abev3',
 'aeri3',
 'aesb3',
 'aflt3',
 'agro3',
 'agxy3',
 'aheb3',
 'aheb5',
 'aheb6',
 'alld3',
 'alpa3',
 'alpa4',
 'alpk3',
 'also3',
 'alup11',
 'alup3',
 'alup4',
 'amar3',
 'ambp3',
 'amer3',
 'anim3',
 'aper3',
 'apti4',
 'arml3',
 'arzz3',
 'asai3',
 'atmp3',
 'atom3',
 'aure3',
 'avll3',
 'azev3',
 'azev4',
 'azul4',
 'b3sa3',
 'bahi3',
 'balm3',
 'balm4',
 'bauh4',
 'baza3',
 'bbas11',
 'bbas3',
 'bbdc3',
 'bbdc4',
 'bbse3',
 'bdll3',
 'bdll4',
 'beef3',
 'bees3',
 'bees4',
 'bgip3',
 'bgip4',
 'bidi11',
 'bidi12',
 'bidi3',
 'bidi4',
 'biom3',
 'bkbr3',
 'blau3',
 'blut3',
 'blut4',
 'bmeb3',
 'bmeb4',
 'bmgb4',
 'bmin3',
 'bmin4',
 'bmks3',
 'bmob3',
 'bnbr3',
 'boas3',
 'bobr3',
 'bobr4',
 'bpac11',
 'bpac3',
 'bpac5',
 'bpan4',
 'bpar3',
 'brap3',
 'brap4',
 'brbi11',
 'brfs3',
 'brge11',
 'brge12',
 'brge3',
 'brge5',
 'brge6',
 'brge7',
 'brge8',
 'brit3',
 'briv3',
 'briv4',
 'brkm3',
 'brkm5',
 'brkm6',
 'brml3',
 'brpr3',
 'brsr3',
 'brsr5',
 '

## Funções

### Verificando se o tick é válido

In [4]:
def validando_tick(tick):
    url_base = f'https://statusinvest.com.br/acoes/{tick}'

    headers = {
        'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.54'
    }

    response = str(requests.get(url_base, headers = headers))
    
    if response == '<Response [200]>':
        return True
    else:
        return False

In [5]:
#Testando a função
print(validando_tick('abev3'))

True


### Substituindo caracteres indesejados

In [6]:
dic = {
    '.': '',
    ',': '.',
    '%': '',
}

In [7]:
def substituir_caracter(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text.strip()

## Abrindo a página

```python
url_base = f'https://statusinvest.com.br/acoes/{tick}'

headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.54'
}

response_text = requests.get(url_base, headers = headers).text

soup = BeautifulSoup(response_text, 'lxml')
```

## Coletando dados gerais da empresa

```python 
nome_empresa = soup.find('h4', {'class': 'mb-2 mt-0 fs-4'}).span.text
cnpj = soup.find('h4', {'class': 'mb-2 mt-0 fs-4'}).small.text

segmentos = soup.find('div', {'class' : 'card bg-main-gd-h white-text rounded ov-hidden pt-0 pb-'}).find_all('strong',{'class':'value'})

setor_atuacao = segmentos[0].text
subsetor_atuacao = segmentos[1].text
segmento_atuacao = segmentos[2].text

valores = soup.find('div', {'class': 'top-info info-3 sm d-flex justify-between mb-3'}).find_all('strong', {'class' : 'value'})

patrimonio_liquido = (valores[0].text.replace('.','').replace(',','.').strip())
ativos = (valores[1].text.replace('.','').replace(',','.').strip())
ativo_circulante = (valores[2].text.replace('.','').replace(',','.').strip())
divida_bruta = (valores[3].text.replace('.','').replace(',','.').strip())
disponibilidade = (valores[4].text.replace('.','').replace(',','.').strip())
divida_liquida = (valores[5].text.replace('.','').replace(',','.').strip())
valor_de_mercado = (valores[6].text.replace('.','').replace(',','.').strip())
valor_de_firma = (valores[7].text.replace('.','').replace(',','.').strip())
num_total_papeis = (valores[8].text.replace('.','').replace(',','.').strip())
segmento_listagem = valores[9].text
free_float = (valores[10].text.replace('%','').replace(',','.').strip())
```

Deixando o código menos repetitivo

```python

valores = soup.find('div', {'class': 'top-info info-3 sm d-flex justify-between mb-3'}).find_all('strong', {'class' : 'value'})

dados_gerais_empresa = ['patrimonio_liquido', 'ativos', 'ativo_circulante', 'divida_bruta', 'disponibilidade', 'divida_liquida', 'valor_de_mercado', 'valor_de_firma', 'num_total_papeis', 'segmento_listagem', 'free_float']

dic_empresa = {}

for i in range(len(dados_gerais_empresa)):
    dic_empresa[dados_gerais_empresa[i]] = substituir_caracter(valores[i].text, dic)
```

## Coletando dados gerais do papel

```python
info_preco = soup.find('div', {'class': 'pb-3 pb-md-5'}).find_all('strong', {'class':'value'})

cotacao_atual = (info_preco[0].text.replace(',','.'))
min_52_sem = (info_preco[1].text.replace(',','.'))
max_52_sem = (info_preco[2].text.replace(',','.'))
dividend_yield = (info_preco[3].text.replace(',','.').replace('%','').replace('-',''))
valorizacao = (info_preco[4].text.replace(',','.').replace('%',''))

```

Deixando o código menos repetitivo
```python
dados_gerais_papel = ['cotacao_atual', 'min_52_sem', 'max_52_sem', 'dividend_yield', 'valorizacao']
dic_papel = {}

for i in range(len(dados_gerais_papel)):
    dic_papel[dados_gerais_papel[i]] = substituir_caracter(info_preco[i].text, dic)
```

## Consolidando em uma função

In [8]:
def coletando_info(tick):
    
    # Dicionário de caracteres para serem substituidos na função substituir_caracter
    sub_caracteres = {
    '.': '',
    ',': '.',
    '%': '',
    }
    
    if validando_tick(tick):
        
        url_base = f'https://statusinvest.com.br/acoes/{tick}'

        headers = {
            'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.54'
        }

        response_text = requests.get(url_base, headers = headers).text

        soup = BeautifulSoup(response_text, 'lxml')

        segmentos = soup.find('div', {'class' : 'card bg-main-gd-h white-text rounded ov-hidden pt-0 pb-0'}).find_all('strong',{'class':'value'})
        valores = soup.find('div', {'class': 'top-info info-3 sm d-flex justify-between mb-3'}).find_all('strong', {'class' : 'value'})    
        info_preco = soup.find('div', {'class': 'pb-3 pb-md-5'}).find_all('strong', {'class':'value'})
        
        info = {'Papel': tick }
        
        info['Empresa'] = soup.find('h4', {'class': 'mb-2 mt-0 fs-4'}).span.text
        info['CNPJ'] = soup.find('h4', {'class': 'mb-2 mt-0 fs-4'}).small.text
        
        dados_segmentos = ['Setor de atuação', 'Subsetor de atuação', 'Segmento de atuação']
                    
        dados_gerais_empresa = ['Patrimônio líquido', 'Ativos', 'Ativo circulante', 
                                'Dívida bruta', 'Disponibilidade', 'Dívida líquida', 
                                'Valor de mercado', 'Valor de firma', 'Número total de papéis',
                                'Segmento de listagem', 'Free float']
        
        dados_gerais_papel = ['Cotação atual', 'Mínima 52 semanas', 'Máxima 52 semanas', 
                              'Dividend Yield', 'Valorização']
       
        for i in range(len(dados_segmentos)):
            info[dados_segmentos[i]] = segmentos[i].text
              
        for i in range(len(dados_gerais_empresa)):
            info[dados_gerais_empresa[i]] = substituir_caracter(valores[i].text, sub_caracteres)
        
        for i in range(len(dados_gerais_papel)):
            info[dados_gerais_papel[i]] = substituir_caracter(info_preco[i].text, sub_caracteres)
        
        return info

In [9]:
# Testando a função

dados = coletando_info('abev3')
print(dados)

{'Papel': 'abev3', 'Empresa': 'AMBEV S.A.', 'CNPJ': '07.526.557/0001-00', 'Setor de atuação': 'Consumo não Cíclico', 'Subsetor de atuação': 'Bebidas', 'Segmento de atuação': 'Cervejas e Refrigerantes', 'Patrimônio líquido': '90178032000', 'Ativos': '142063954000', 'Ativo circulante': '41556965000', 'Dívida bruta': '2980480000', 'Disponibilidade': '19059870000', 'Dívida líquida': '-16079390000', 'Valor de mercado': '225543105306', 'Valor de firma': '209463715306', 'Número total de papéis': '15750216851', 'Segmento de listagem': '', 'Free float': '27.85', 'Cotação atual': '14.32', 'Mínima 52 semanas': '12.47', 'Máxima 52 semanas': '16.03', 'Dividend Yield': '5.32', 'Valorização': '2.95'}


## Coletando dados de múltiplas empresas

In [10]:
lista_consolidada = []

start_time = time.time()

for tick in ticks:
    dados = coletando_info(tick)
    if dados != None:
        lista_consolidada.append(dados)
        
print(f"Tempo de execução: {round(time.time() - start_time, 4)} segundos")

Tempo de execução: 450.107 segundos


In [11]:
lista_consolidada

[{'Papel': 'aalr3',
  'Empresa': 'CENTRO DE IMAGEM DIAGNOSTICOS S.A.',
  'CNPJ': '42.771.949/0001-35',
  'Setor de atuação': 'Saúde',
  'Subsetor de atuação': 'Serv.Méd.Hospit..Análises e Diagnósticos',
  'Segmento de atuação': 'Serv.Méd.Hospit..Análises e Diagnósticos',
  'Patrimônio líquido': '1145368000',
  'Ativos': '2573833000',
  'Ativo circulante': '457827000',
  'Dívida bruta': '808798000',
  'Disponibilidade': '240190000',
  'Dívida líquida': '568608000',
  'Valor de mercado': '2518454053',
  'Valor de firma': '3087062053',
  'Número total de papéis': '118292816',
  'Segmento de listagem': 'Novo Mercado',
  'Free float': '35.52',
  'Cotação atual': '21.29',
  'Mínima 52 semanas': '12.85',
  'Máxima 52 semanas': '21.60',
  'Dividend Yield': '-',
  'Valorização': '65.68'},
 {'Papel': 'abcb4',
  'Empresa': 'BCO ABC BRASIL S.A.',
  'CNPJ': '28.195.667/0001-06',
  'Setor de atuação': 'Financeiro e Outros',
  'Subsetor de atuação': 'Intermediários Financeiros',
  'Segmento de atuaçã

## Criando o Dataframe

In [12]:
df_statusinvest = pd.DataFrame(lista_consolidada)

In [13]:
df_statusinvest.head()

,Papel,Empresa,CNPJ,Setor de atuação,Subsetor de atuação,Segmento de atuação,Patrimônio líquido,Ativos,Ativo circulante,Dívida bruta,...,Valor de mercado,Valor de firma,Número total de papéis,Segmento de listagem,Free float,Cotação atual,Mínima 52 semanas,Máxima 52 semanas,Dividend Yield,Valorização
0,aalr3,CENTRO DE IMAGEM DIAGNOSTICOS S.A.,42.771.949/0001-35,Saúde,Serv.Méd.Hospit..Análises e Diagnósticos,Serv.Méd.Hospit..Análises e Diagnósticos,1145368000,2573833000,457827000,808798000,...,2518454053,3087062053,118292816,Novo Mercado,35.52,21.29,12.85,21.60,-,65.68
1,abcb4,BCO ABC BRASIL S.A.,28.195.667/0001-06,Financeiro e Outros,Intermediários Financeiros,Bancos,5070725000,52140478000,4297215000,-,...,4291190440,4291190440,226090118,Nível 2,29.88,18.98,14.47,22.29,7.36,30.00
2,abev3,AMBEV S.A.,07.526.557/0001-00,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,90178032000,142063954000,41556965000,2980480000,...,225543105306,209463715306,15750216851,,27.85,14.32,12.47,16.03,5.32,2.95
3,aeri3,AERIS IND. E COM. DE EQUIP. GERACAO DE ENERGIA...,12.528.708/0001-07,Bens Industriais,Máquinas e Equipamentos,Máq. e Equip. Industriais,942087000,3459015000,2285013000,1573577000,...,819848398,1537033398,766213456,Novo Mercado,29.64,1.07,0.88,6.54,1.93,-81.83
4,aesb3,AES BRASIL ENERGIA S.A.,37.663.076/0001-07,Utilidade Pública,Energia Elétrica,Energia Elétrica,4175175000,15104672000,4091868000,7893505000,...,5573846900,10011969900,601927311,Novo Mercado,52.37,9.26,9.05,12.06,1.16,-7.12


In [16]:
df_statusinvest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459 entries, 0 to 458
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Papel                   459 non-null    object
 1   Empresa                 459 non-null    object
 2   CNPJ                    459 non-null    object
 3   Setor de atuação        459 non-null    object
 4   Subsetor de atuação     459 non-null    object
 5   Segmento de atuação     459 non-null    object
 6   Patrimônio líquido      459 non-null    object
 7   Ativos                  459 non-null    object
 8   Ativo circulante        459 non-null    object
 9   Dívida bruta            459 non-null    object
 10  Disponibilidade         459 non-null    object
 11  Dívida líquida          459 non-null    object
 12  Valor de mercado        459 non-null    object
 13  Valor de firma          459 non-null    object
 14  Número total de papéis  459 non-null    object
 15  Segmen

In [15]:
df_statusinvest.to_csv('dados_empresas_statsusinvest.csv', sep = ';', encoding = 'UTF-8', index = False)